In [1]:
import redis
import json

# Define connection variables
host = 'redis-13857.c14.us-east-1-2.ec2.redns.redis-cloud.com'
port = 13857
password = 'okNaqK7r5dKE6stSis0FlUuiDlhH2t7M'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Post-purchase Feedback Collection

## Description of the use case
This use case involves collecting feedback from customers after a purchase to gain insights into their satisfaction and areas for improvement. The feedback can be used to enhance products and services.

## Redis Data Structures Used
- **Hashes**: To store feedback information.
- **Streams**: To capture and process real-time feedback submissions.
- **Sorted Sets**: To rank feedback based on ratings or urgency.

In [2]:
# Mock data for the use case
mock_data = {
    'feedback_1001': {'order_id': 'order_1001', 'customer_id': 'cust_1'},
    'feedback_1002': {'order_id': 'order_1002', 'customer_id': 'cust_2'},
    'feedback_1003': {'order_id': 'order_1003', 'customer_id': 'cust_3'},
}

# Use Hashes to store feedback information
for feedback_id, feedback_info in mock_data.items():
    r.hset(feedback_id, mapping=feedback_info)

# Use Streams to capture real-time feedback submissions
r.xadd('feedback_stream', {'feedback_id': 'feedback_1001', 'rating': 4, 'comment': 'Great product!'})
r.xadd('feedback_stream', {'feedback_id': 'feedback_1002', 'rating': 5, 'comment': 'Excellent service!'})

# Function to process feedback submissions
def process_feedback_submissions():
    stream_entries = r.xrange('feedback_stream')
    for ts, entry in stream_entries:
        feedback_id = entry['feedback_id']
        rating = int(entry['rating'])
        comment = entry['comment']
        r.hset(feedback_id, 'rating', rating)
        r.hset(feedback_id, 'comment', comment)

# Process the feedback submissions
print(f"Processed Feedback:")
process_feedback_submissions()

# Use Sorted Sets to rank feedback based on ratings
for feedback_id in mock_data.keys():
    if r.hget(feedback_id, 'rating'):
        rating = int(r.hget(feedback_id, 'rating'))
        r.zadd('feedback_ranking', {feedback_id: rating})
    else:
        print(f"No feedback recieved for feedback id : {feedback_id}")

# Print the collected feedback information
for feedback_id in mock_data.keys():
    print(f"{feedback_id}: {r.hgetall(feedback_id)}")

# Print the ranked feedback
ranked_feedback = r.zrange('feedback_ranking', 0, -1, withscores=True, desc=True)
print('Ranked Feedback:')
for feedback in ranked_feedback:
    print(feedback)
    
print(f"Top feedback :")
f_id, _ = r.zrevrange('feedback_ranking', 0, 0 , withscores=True)[0]
o_id = r.hgetall(f_id)
print(o_id)

Processed Feedback:
No feedback recieved for feedback id : feedback_1003
feedback_1001: {'order_id': 'order_1001', 'customer_id': 'cust_1', 'rating': '4', 'comment': 'Great product!'}
feedback_1002: {'order_id': 'order_1002', 'customer_id': 'cust_2', 'rating': '5', 'comment': 'Excellent service!'}
feedback_1003: {'order_id': 'order_1003', 'customer_id': 'cust_3'}
Ranked Feedback:
('feedback_1002', 5.0)
('feedback_1001', 4.0)
Top feedback :
{'order_id': 'order_1002', 'customer_id': 'cust_2', 'rating': '5', 'comment': 'Excellent service!'}
